In [17]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import networkx as nx
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
%matplotlib inline

In [18]:
# Importing the text the lazy way.
gatsby = "In my younger and more vulnerable years my father gave me some advice that I've been turning over in my mind ever since. \"Whenever you feel like criticizing any one,\" he told me, \"just remember that all the people in this world haven't had the advantages that you've had.\" He didn't say any more but we've always been unusually communicative in a reserved way, and I understood that he meant a great deal more than that. In consequence I'm inclined to reserve all judgments, a habit that has opened up many curious natures to me and also made me the victim of not a few veteran bores. The abnormal mind is quick to detect and attach itself to this quality when it appears in a normal person, and so it came about that in college I was unjustly accused of being a politician, because I was privy to the secret griefs of wild, unknown men. Most of the confidences were unsought--frequently I have feigned sleep, preoccupation, or a hostile levity when I realized by some unmistakable sign that an intimate revelation was quivering on the horizon--for the intimate revelations of young men or at least the terms in which they express them are usually plagiaristic and marred by obvious suppressions. Reserving judgments is a matter of infinite hope. I am still a little afraid of missing something if I forget that, as my father snobbishly suggested, and I snobbishly repeat a sense of the fundamental decencies is parcelled out unequally at birth. And, after boasting this way of my tolerance, I come to the admission that it has a limit. Conduct may be founded on the hard rock or the wet marshes but after a certain point I don't care what it's founded on. When I came back from the East last autumn I felt that I wanted the world to be in uniform and at a sort of moral attention forever; I wanted no more riotous excursions with privileged glimpses into the human heart. Only Gatsby, the man who gives his name to this book, was exempt from my reaction--Gatsby who represented everything for which I have an unaffected scorn. If personality is an unbroken series of successful gestures, then there was something gorgeous about him, some heightened sensitivity to the promises of life, as if he were related to one of those intricate machines that register earthquakes ten thousand miles away. This responsiveness had nothing to do with that flabby impressionability which is dignified under the name of the \"creative temperament\"--it was an extraordinary gift for hope, a romantic readiness such as I have never found in any other person and which it is not likely I shall ever find again. No--Gatsby turned out all right at the end; it is what preyed on Gatsby, what foul dust floated in the wake of his dreams that temporarily closed out my interest in the abortive sorrows and short-winded elations of men."

# We want to use the standard english-language parser.
parser = spacy.load('en')

In [19]:
gatsby = parser(gatsby)
# make sentences
sentences = []
for span in gatsby.sents:
    sent = ' '.join(gatsby[i].string for i in range(span.start, span.end)).strip()
    sentences.append(sent)
# creating td-idf matrix
counter = TfidfVectorizer(lowercase=False,
                          stop_words=None,
                         ngram_range=(1, 1),
                         analyzer=u'word',
                         max_df=0.5,
                         min_df=1,
                         max_features=None,
                         vocabulary=None,
                         binary=False)
# apply vectorizer
data_counts=counter.fit_transform(sentences)

We now have sentences represented as vectors, with each word having a score based on how often it occurs in the sentence divided by how often it occurs in the whole text.

Now let's calculate the similarity scores for the sentences and apply the TextRank algorithm.

In [20]:
# Calculating similarity
similarity = data_counts * data_counts.T

# identify sentence with highest rank
nx_graph = nx.from_scipy_sparse_matrix(similarity)
ranks = nx.pagerank(nx_graph, alpha=0.85, tol=0.00000001)
ranked = sorted(((ranks[i],s) for i,s in enumerate(sentences)), reverse=True)
print(ranked[0])

(0.07580280938678051, 'This  responsiveness  had  nothing  to  do  with  that  flabby  impressionability  which  is  dignified  under  the  name  of  the  " creative  temperament "-- it  was  an  extraordinary  gift  for  hope ,  a  romantic  readiness  such  as  I  have  never  found  in  any  other  person  and  which  it  is  not  likely  I  shall  ever  find  again .')


In [21]:
# Removing stop words and punctuation, then getting a list of all unique words in the text
gatsby_filt = [word for word in gatsby if word.is_stop==False and (word.pos_=='NOUN' or word.pos_=='ADJ')]
words=set(gatsby_filt)

# set grid
adjacency=pd.DataFrame(columns=words,index=words,data=0)

#iterate
for i,word in enumerate(gatsby): 
    # check if 4 neighbors are in word list
    if any ([word == item for item in gatsby_filt]):
            # stop at end of string
        end=max(0,len(gatsby)-(len(gatsby)-(i+5)))
            # potential neighbors
        nextwords=gatsby[i+1:end]
            # filtering the neighbors to select only ones in word list
             # Filtering the neighbors to select only those in the word list
        inset=[x in gatsby_filt for x in nextwords]
        neighbors=[nextwords[i] for i in range(len(nextwords)) if inset[i]]
        # Adding 1 to the adjacency matrix for neighbors of the target word
        if neighbors:
            adjacency.loc[word,neighbors]=adjacency.loc[word,neighbors]+1

print('done!')

done!


In [22]:
# Running TextRank
nx_words = nx.from_numpy_matrix(adjacency.as_matrix())
ranks=nx.pagerank(nx_words, alpha=.85, tol=.00000001)

# Identifying the five most highly ranked keywords
ranked = sorted(((ranks[i],s) for i,s in enumerate(words)),
                reverse=True)
print(ranked[:5])


[(0.013370948308795436, hope), (0.012223431176324349, promises), (0.012223431176324349, exempt), (0.012142068850548908, glimpses), (0.011895137937387881, intimate)]


It is also possible that keyword phrases will work better. Modfiy the keyword extraction code to extract two-word phrases (digrams) rather than single words. Then try it with trigrams. You will probably want to broaden the window that defines 'neighbors.' Try a few different modifications, and write up your observations in your notebook. Discuss with your mentor.

# Digram with more neighbors

In [29]:
# Importing the text the lazy way.
gatsby="In my younger and more vulnerable years my father gave me some advice that I've been turning over in my mind ever since. \"Whenever you feel like criticizing any one,\" he told me, \"just remember that all the people in this world haven't had the advantages that you've had.\" He didn't say any more but we've always been unusually communicative in a reserved way, and I understood that he meant a great deal more than that. In consequence I'm inclined to reserve all judgments, a habit that has opened up many curious natures to me and also made me the victim of not a few veteran bores. The abnormal mind is quick to detect and attach itself to this quality when it appears in a normal person, and so it came about that in college I was unjustly accused of being a politician, because I was privy to the secret griefs of wild, unknown men. Most of the confidences were unsought--frequently I have feigned sleep, preoccupation, or a hostile levity when I realized by some unmistakable sign that an intimate revelation was quivering on the horizon--for the intimate revelations of young men or at least the terms in which they express them are usually plagiaristic and marred by obvious suppressions. Reserving judgments is a matter of infinite hope. I am still a little afraid of missing something if I forget that, as my father snobbishly suggested, and I snobbishly repeat a sense of the fundamental decencies is parcelled out unequally at birth. And, after boasting this way of my tolerance, I come to the admission that it has a limit. Conduct may be founded on the hard rock or the wet marshes but after a certain point I don't care what it's founded on. When I came back from the East last autumn I felt that I wanted the world to be in uniform and at a sort of moral attention forever; I wanted no more riotous excursions with privileged glimpses into the human heart. Only Gatsby, the man who gives his name to this book, was exempt from my reaction--Gatsby who represented everything for which I have an unaffected scorn. If personality is an unbroken series of successful gestures, then there was something gorgeous about him, some heightened sensitivity to the promises of life, as if he were related to one of those intricate machines that register earthquakes ten thousand miles away. This responsiveness had nothing to do with that flabby impressionability which is dignified under the name of the \"creative temperament\"--it was an extraordinary gift for hope, a romantic readiness such as I have never found in any other person and which it is not likely I shall ever find again. No--Gatsby turned out all right at the end; it is what preyed on Gatsby, what foul dust floated in the wake of his dreams that temporarily closed out my interest in the abortive sorrows and short-winded elations of men."

# We want to use the standard english-language parser.
parser = spacy.load('en')

# Parsing Gatsby.
gatsby = parser(gatsby)

# Dividing the text into sentences and storing them as a list of strings.
sentences=[]
for span in gatsby.sents:
    # go from the start to the end of each span, returning each token in the sentence
    # combine each token using join()
    sent = ''.join(gatsby[i].string for i in range(span.start, span.end)).strip()
    sentences.append(sent)

# Creating the tf-idf matrix.
counter = TfidfVectorizer(lowercase=True, 
                          stop_words=None,
                          ngram_range=(2, 2), 
                          analyzer=u'word', 
                          max_df=.5, 
                          min_df=1,
                          max_features=None, 
                          vocabulary=None, 
                          binary=False)

#Applying the vectorizer
data_counts=counter.fit_transform(sentences)

In [30]:
counter.vocabulary_
inv_map = {v: k for k, v in counter.vocabulary_.items()}
vocab_ordered = [inv_map[i] for i in range(0, len(inv_map))]
counts_df = pd.DataFrame(data_counts.toarray(), columns=vocab_ordered)
counts_df

,abnormal mind,abortive sorrows,about him,about that,accused of,admission that,advantages that,advice that,afraid of,after boasting,...,winded elations,with privileged,with that,world haven,world to,years my,you feel,you ve,young men,younger and
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.205159,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.205159,0.000000,0.000000,0.00000,0.205159
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.196116,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.196116,0.000000,0.000000,0.196116,0.196116,0.00000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
4,0.197964,0.000000,0.000000,0.197964,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.253126,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.13819,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.19116,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.261647,0.000000,0.000000,0.00000,0.261647,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000


In [31]:
# Calculating similarity
similarity = data_counts * data_counts.T

# identify sentence with highest rank
nx_graph = nx.from_scipy_sparse_matrix(similarity)
ranks = nx.pagerank(nx_graph, alpha=0.85, tol=0.00000001)
ranked = sorted(((ranks[i],s) for i,s in enumerate(sentences)), reverse=True)
print(ranked[0])

(0.06288604982651592, 'This responsiveness had nothing to do with that flabby impressionability which is dignified under the name of the "creative temperament"--it was an extraordinary gift for hope, a romantic readiness such as I have never found in any other person and which it is not likely I shall ever find again.')


In [ ]:
# Running TextRank
nx_words = nx.from_numpy_matrix(adjacency.as_matrix())
ranks=nx.pagerank(nx_words, alpha=.85, tol=.00000001)

# Identifying the five most highly ranked keywords
ranked = sorted(((ranks[i],s) for i,s in enumerate(words)),
                reverse=True)
print(ranked[:7])

In [32]:
import string
string.punctuation
spacy_stopwords = list(spacy.lang.en.stop_words.STOP_WORDS)
gatsby="In my younger and more vulnerable years my father gave me some advice that I've been turning over in my mind ever since. \"Whenever you feel like criticizing any one,\" he told me, \"just remember that all the people in this world haven't had the advantages that you've had.\" He didn't say any more but we've always been unusually communicative in a reserved way, and I understood that he meant a great deal more than that. In consequence I'm inclined to reserve all judgments, a habit that has opened up many curious natures to me and also made me the victim of not a few veteran bores. The abnormal mind is quick to detect and attach itself to this quality when it appears in a normal person, and so it came about that in college I was unjustly accused of being a politician, because I was privy to the secret griefs of wild, unknown men. Most of the confidences were unsought--frequently I have feigned sleep, preoccupation, or a hostile levity when I realized by some unmistakable sign that an intimate revelation was quivering on the horizon--for the intimate revelations of young men or at least the terms in which they express them are usually plagiaristic and marred by obvious suppressions. Reserving judgments is a matter of infinite hope. I am still a little afraid of missing something if I forget that, as my father snobbishly suggested, and I snobbishly repeat a sense of the fundamental decencies is parcelled out unequally at birth. And, after boasting this way of my tolerance, I come to the admission that it has a limit. Conduct may be founded on the hard rock or the wet marshes but after a certain point I don't care what it's founded on. When I came back from the East last autumn I felt that I wanted the world to be in uniform and at a sort of moral attention forever; I wanted no more riotous excursions with privileged glimpses into the human heart. Only Gatsby, the man who gives his name to this book, was exempt from my reaction--Gatsby who represented everything for which I have an unaffected scorn. If personality is an unbroken series of successful gestures, then there was something gorgeous about him, some heightened sensitivity to the promises of life, as if he were related to one of those intricate machines that register earthquakes ten thousand miles away. This responsiveness had nothing to do with that flabby impressionability which is dignified under the name of the \"creative temperament\"--it was an extraordinary gift for hope, a romantic readiness such as I have never found in any other person and which it is not likely I shall ever find again. No--Gatsby turned out all right at the end; it is what preyed on Gatsby, what foul dust floated in the wake of his dreams that temporarily closed out my interest in the abortive sorrows and short-winded elations of men."
gatsby_words = gatsby.split(' ')
gatsby_words = [g.lower().strip(string.punctuation) for g in gatsby_words]
gatsby_words = [g for g in gatsby_words if g not in spacy_stopwords]

In [34]:
gatsby_bigram = [gatsby_words[i] + ',' + gatsby_words[i+1] for i in range(0, len(gatsby_words)-1)]
gatsby_bigram_set = set(gatsby_bigram)

adjacency=pd.DataFrame(columns=gatsby_bigram_set,index=gatsby_bigram_set,data=0)

#Iterating through each word in the text and indicating which of the unique words are its neighbors
for i,word in enumerate(gatsby_bigram):
    # Checking if any of the word's next four neighbors are in the word list 
    if any([word == item for item in gatsby_bigram_set]):
        # Making sure to stop at the end of the string, even if there are less than four words left after the target.
        end=max(0,len(gatsby_bigram)-(len(gatsby_bigram)-(i+6)))
        # The potential neighbors.
        nextwords=gatsby_bigram[i+1:end]
        # Filtering the neighbors to select only those in the word list
        inset=[x in gatsby_bigram_set for x in nextwords]
        neighbors=[nextwords[i] for i in range(len(nextwords)) if inset[i]]
        # Adding 1 to the adjacency matrix for neighbors of the target word
        if neighbors:
                adjacency.loc[word,neighbors]=adjacency.loc[word,neighbors]+1

print('done!')

# Running TextRank
nx_words = nx.from_numpy_matrix(adjacency.as_matrix())
ranks=nx.pagerank(nx_words, alpha=.85, tol=.00000001)

# Identifying the five most highly ranked keywords
ranked = sorted(((ranks[i],s) for i,s in enumerate(gatsby_bigram_set)),
                reverse=True)
print(ranked[:5])


done!
[(0.005541748086646039, 'closed,interest'), (0.005541748086646039, "advice,i've"), (0.005440919072300158, 'temporarily,closed'), (0.005440919072300158, "i've,turning"), (0.005358010337055437, 'turning,mind')]


# Trigram

In [36]:
# Importing the text the lazy way.
gatsby="In my younger and more vulnerable years my father gave me some advice that I've been turning over in my mind ever since. \"Whenever you feel like criticizing any one,\" he told me, \"just remember that all the people in this world haven't had the advantages that you've had.\" He didn't say any more but we've always been unusually communicative in a reserved way, and I understood that he meant a great deal more than that. In consequence I'm inclined to reserve all judgments, a habit that has opened up many curious natures to me and also made me the victim of not a few veteran bores. The abnormal mind is quick to detect and attach itself to this quality when it appears in a normal person, and so it came about that in college I was unjustly accused of being a politician, because I was privy to the secret griefs of wild, unknown men. Most of the confidences were unsought--frequently I have feigned sleep, preoccupation, or a hostile levity when I realized by some unmistakable sign that an intimate revelation was quivering on the horizon--for the intimate revelations of young men or at least the terms in which they express them are usually plagiaristic and marred by obvious suppressions. Reserving judgments is a matter of infinite hope. I am still a little afraid of missing something if I forget that, as my father snobbishly suggested, and I snobbishly repeat a sense of the fundamental decencies is parcelled out unequally at birth. And, after boasting this way of my tolerance, I come to the admission that it has a limit. Conduct may be founded on the hard rock or the wet marshes but after a certain point I don't care what it's founded on. When I came back from the East last autumn I felt that I wanted the world to be in uniform and at a sort of moral attention forever; I wanted no more riotous excursions with privileged glimpses into the human heart. Only Gatsby, the man who gives his name to this book, was exempt from my reaction--Gatsby who represented everything for which I have an unaffected scorn. If personality is an unbroken series of successful gestures, then there was something gorgeous about him, some heightened sensitivity to the promises of life, as if he were related to one of those intricate machines that register earthquakes ten thousand miles away. This responsiveness had nothing to do with that flabby impressionability which is dignified under the name of the \"creative temperament\"--it was an extraordinary gift for hope, a romantic readiness such as I have never found in any other person and which it is not likely I shall ever find again. No--Gatsby turned out all right at the end; it is what preyed on Gatsby, what foul dust floated in the wake of his dreams that temporarily closed out my interest in the abortive sorrows and short-winded elations of men."

# We want to use the standard english-language parser.
parser = spacy.load('en')

# Parsing Gatsby.
gatsby = parser(gatsby)

# Dividing the text into sentences and storing them as a list of strings.
sentences=[]
for span in gatsby.sents:
    # go from the start to the end of each span, returning each token in the sentence
    # combine each token using join()
    sent = ''.join(gatsby[i].string for i in range(span.start, span.end)).strip()
    sentences.append(sent)

# Creating the tf-idf matrix.
counter = TfidfVectorizer(lowercase=True, 
                          stop_words=None,
                          ngram_range=(3, 3), 
                          analyzer=u'word', 
                          max_df=.75, 
                          min_df=1,
                          max_features=None, 
                          vocabulary=None, 
                          binary=False)

#Applying the vectorizer
data_counts=counter.fit_transform(sentences)

In [37]:
counter.vocabulary_
inv_map = {v: k for k, v in counter.vocabulary_.items()}
vocab_ordered = [inv_map[i] for i in range(0, len(inv_map))]
counts_df = pd.DataFrame(data_counts.toarray(), columns=vocab_ordered)
counts_df

,abnormal mind is,abortive sorrows and,about him some,about that in,accused of being,admission that it,advantages that you,advice that ve,afraid of missing,after boasting this,...,winded elations of,with privileged glimpses,with that flabby,world haven had,world to be,years my father,you feel like,you ve had,young men or,younger and more
0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.218218,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.218218,0.0,0.0,0.000000,0.218218
1,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.2,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.2,0.2,0.000000,0.000000
2,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
3,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
4,0.2,0.000000,0.000000,0.2,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
5,0.0,0.000000,0.000000,0.0,0.258199,0.000000,0.0,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
6,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.138675,0.000000
7,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
8,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.19245,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
9,0.0,0.000000,0.000000,0.0,0.000000,0.267261,0.0,0.000000,0.00000,0.267261,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000


In [38]:
# Calculating similarity
similarity = data_counts * data_counts.T

# data_counts is size (# sentences, # vocab) --> # of sentences x # of sentences

# Identifying the sentence with the highest rank.
nx_graph = nx.from_scipy_sparse_matrix(similarity)
ranks=nx.pagerank(nx_graph, alpha=.85, tol=.00000001)

ranked = sorted(((ranks[i],s) for i,s in enumerate(sentences)),
                reverse=True)
print(ranked[0])

(0.0625, 'When I came back from the East last autumn I felt that I wanted the world to be in uniform and at a sort of moral attention forever; I wanted no more riotous excursions with privileged glimpses into the human heart.')


In [40]:
import string
string.punctuation
spacy_stopwords = list(spacy.lang.en.stop_words.STOP_WORDS)
gatsby="In my younger and more vulnerable years my father gave me some advice that I've been turning over in my mind ever since. \"Whenever you feel like criticizing any one,\" he told me, \"just remember that all the people in this world haven't had the advantages that you've had.\" He didn't say any more but we've always been unusually communicative in a reserved way, and I understood that he meant a great deal more than that. In consequence I'm inclined to reserve all judgments, a habit that has opened up many curious natures to me and also made me the victim of not a few veteran bores. The abnormal mind is quick to detect and attach itself to this quality when it appears in a normal person, and so it came about that in college I was unjustly accused of being a politician, because I was privy to the secret griefs of wild, unknown men. Most of the confidences were unsought--frequently I have feigned sleep, preoccupation, or a hostile levity when I realized by some unmistakable sign that an intimate revelation was quivering on the horizon--for the intimate revelations of young men or at least the terms in which they express them are usually plagiaristic and marred by obvious suppressions. Reserving judgments is a matter of infinite hope. I am still a little afraid of missing something if I forget that, as my father snobbishly suggested, and I snobbishly repeat a sense of the fundamental decencies is parcelled out unequally at birth. And, after boasting this way of my tolerance, I come to the admission that it has a limit. Conduct may be founded on the hard rock or the wet marshes but after a certain point I don't care what it's founded on. When I came back from the East last autumn I felt that I wanted the world to be in uniform and at a sort of moral attention forever; I wanted no more riotous excursions with privileged glimpses into the human heart. Only Gatsby, the man who gives his name to this book, was exempt from my reaction--Gatsby who represented everything for which I have an unaffected scorn. If personality is an unbroken series of successful gestures, then there was something gorgeous about him, some heightened sensitivity to the promises of life, as if he were related to one of those intricate machines that register earthquakes ten thousand miles away. This responsiveness had nothing to do with that flabby impressionability which is dignified under the name of the \"creative temperament\"--it was an extraordinary gift for hope, a romantic readiness such as I have never found in any other person and which it is not likely I shall ever find again. No--Gatsby turned out all right at the end; it is what preyed on Gatsby, what foul dust floated in the wake of his dreams that temporarily closed out my interest in the abortive sorrows and short-winded elations of men."
gatsby_words = gatsby.split(' ')
gatsby_words = [g.lower().strip(string.punctuation) for g in gatsby_words]
gatsby_words = [g for g in gatsby_words if g not in spacy_stopwords]
#TODO: STRIP OUT STOP WORDS FROM GATSBY_WORDS
gatsby_bigram = [gatsby_words[i] + ',' + gatsby_words[i+1] + ',' + gatsby_words[i+2] for i in range(0, len(gatsby_words)-2)]
gatsby_bigram_set = set(gatsby_bigram)

adjacency=pd.DataFrame(columns=gatsby_bigram_set,index=gatsby_bigram_set,data=0)

#Iterating through each word in the text and indicating which of the unique words are its neighbors
for i,word in enumerate(gatsby_bigram):
    # Checking if any of the word's next four neighbors are in the word list 
    if any([word == item for item in gatsby_bigram_set]):
        # Making sure to stop at the end of the string, even if there are less than four words left after the target.
        end=max(0,len(gatsby_bigram)-(len(gatsby_bigram)-(i+5)))
        # The potential neighbors.
        nextwords=gatsby_bigram[i+1:end]
        # Filtering the neighbors to select only those in the word list
        inset=[x in gatsby_bigram_set for x in nextwords]
        neighbors=[nextwords[i] for i in range(len(nextwords)) if inset[i]]
        # Adding 1 to the adjacency matrix for neighbors of the target word
        if neighbors:
                adjacency.loc[word,neighbors]=adjacency.loc[word,neighbors]+1

print('done!')

# Running TextRank
nx_words = nx.from_numpy_matrix(adjacency.as_matrix())
ranks=nx.pagerank(nx_words, alpha=.85, tol=.00000001)

# Identifying the five most highly ranked keywords
ranked = sorted(((ranks[i],s) for i,s in enumerate(gatsby_bigram_set)),
                reverse=True)
print(ranked[:5])


done!
[(0.0055826973314369675, "gave,advice,i've"), (0.0055826973314369675, 'closed,interest,abortive'), (0.0054596850863936645, 'temporarily,closed,interest'), (0.0054596850863936645, "advice,i've,turning"), (0.005362364680755169, 'dreams,temporarily,closed')]
